In [1]:
# Standard libraries
import os

# Third-party libraries
import geopandas as gpd
import pandas as pd


In [2]:
DATA_DIR = os.path.join(os.getcwd(), "data")
GEO_DIR = os.path.join(DATA_DIR, "external/spatial")
OUT_DIR = os.path.join(DATA_DIR, "processed")
OSGB36 = 27700


In [3]:
def set_crs(gdf):
    """Assigns the OSGB36 / British National Grid CRS to a GeoDataFrame"""
    return gdf.set_crs(epsg=OSGB36, allow_override=True, inplace=True)


def get_parent_id(child, parents):
    '''Returns the ID of the polygon that contains a point'''
    point = gpd.points_from_xy(x=[child.x_coord], y=[child.y_coord])[0]
    for parent in parents.itertuples():
        if parent.geometry.contains(point):
            return parent.id
    return None


def format(df):
    """Rename some columns and drops some others"""
    return (
        df.rename(columns={"geo_region": "name"})
        .drop(columns=["region"])
        .sort_values(by="name", axis=0)
        .reset_index(drop=True)
        
    )

In [4]:
uk = (
    gpd.read_file(os.path.join(GEO_DIR, "ukcp18-uk-land-country-united_kingdom-hires.geojson"), driver="GeoJSON")
    .pipe(set_crs)
    .pipe(format)
)
countries = (
    gpd.read_file(os.path.join(GEO_DIR, "ukcp18-uk-land-country-hires.geojson"), driver="GeoJSON")
    .pipe(set_crs)
    .pipe(format)
)
regions = (
    gpd.read_file(os.path.join(GEO_DIR, "ukcp18-uk-land-region-hires.geojson"), driver="GeoJSON")
    .pipe(set_crs)
    .pipe(format)
)


ERROR 1: PROJ: proj_create_from_database: Open of /home/mikeblackett/anaconda3/envs/climate-watch-uk/share/proj failed


In [5]:
countries = pd.concat([countries, uk]).sort_values(by="name").reset_index(drop=True)
countries


,name,x_coord,y_coord,geometry
0,Channel Islands,378641.915415,-63617.081409,"MULTIPOLYGON (((388778.830 -70524.923, 387476...."
1,England,436325.368279,296147.848719,"MULTIPOLYGON (((84009.402 5361.604, 84022.099 ..."
2,Isle of Man,234265.323626,484561.023312,"MULTIPOLYGON (((216561.496 466536.198, 216335...."
3,Northern Ireland,97015.507927,533949.890497,"MULTIPOLYGON (((45089.390 487652.021, 44926.19..."
4,Scotland,266651.514962,771689.385884,"MULTIPOLYGON (((336177.917 1014260.958, 336272..."
5,United Kingdom,348890.323001,461087.313870,"MULTIPOLYGON (((336177.917 1014260.958, 336272..."
6,Wales,279739.337741,271715.748248,"MULTIPOLYGON (((322349.697 164637.902, 322360...."


In [6]:
uk.insert(0, 'id', range(1, 1 + len(uk)))
countries.insert(0, 'id', range(1, 1 + len(countries)))
regions.insert(0, 'id', range(1, 1 + len(regions)))

In [12]:
countries["parentId"] = countries.apply(lambda row: get_parent_id(row, uk), axis=1)
countries


,id,name,x_coord,y_coord,geometry,parentId
0,1,Channel Islands,378641.915415,-63617.081409,"MULTIPOLYGON (((388778.830 -70524.923, 387476....",NaN
1,2,England,436325.368279,296147.848719,"MULTIPOLYGON (((84009.402 5361.604, 84022.099 ...",1.0
2,3,Isle of Man,234265.323626,484561.023312,"MULTIPOLYGON (((216561.496 466536.198, 216335....",NaN
3,4,Northern Ireland,97015.507927,533949.890497,"MULTIPOLYGON (((45089.390 487652.021, 44926.19...",1.0
4,5,Scotland,266651.514962,771689.385884,"MULTIPOLYGON (((336177.917 1014260.958, 336272...",1.0
5,6,United Kingdom,348890.323001,461087.313870,"MULTIPOLYGON (((336177.917 1014260.958, 336272...",1.0
6,7,Wales,279739.337741,271715.748248,"MULTIPOLYGON (((322349.697 164637.902, 322360....",1.0


In [8]:
regions["parentId"] = regions.apply(lambda row: get_parent_id(row, countries), axis=1)
regions


,id,name,x_coord,y_coord,geometry,parentId
0,1,Channel Islands,378641.915415,-63617.081409,"MULTIPOLYGON (((388778.830 -70524.923, 387476....",NaN
1,2,East Midlands,481004.614478,337237.029489,"POLYGON ((534657.901 406164.796, 533900.897 40...",2.0
2,3,East Scotland,327936.681532,732569.478746,"MULTIPOLYGON (((320766.200 680581.505, 320697....",5.0
3,4,East of England,573749.537210,263770.914800,"POLYGON ((585951.803 181704.897, 585934.901 18...",2.0
4,5,Isle of Man,234265.323626,484561.023312,"MULTIPOLYGON (((216561.496 466536.198, 216335....",3.0
5,6,London,531331.013781,179645.800174,"POLYGON ((531788.999 200757.503, 531643.597 20...",2.0
6,7,North East England,406477.705975,569475.671027,"MULTIPOLYGON (((429246.996 604972.298, 429197....",2.0
7,8,North Scotland,241176.638224,880259.997210,"MULTIPOLYGON (((172604.496 740747.502, 172569....",5.0
8,9,North West England,351884.161496,461276.950523,"POLYGON ((364028.497 577614.499, 363932.904 57...",2.0
9,10,Northern Ireland,97565.621077,534023.088005,"MULTIPOLYGON (((44261.773 488364.956, 44370.22...",4.0


In [9]:
countries.to_file(os.path.join(OUT_DIR, "country.geojson"), driver="GeoJSON")
countries.drop(columns='geometry').to_csv(os.path.join(OUT_DIR, "country.csv"), index=False)
regions.to_file(os.path.join(OUT_DIR, "region.geojson"), driver="GeoJSON")
regions.drop(columns='geometry').to_csv(os.path.join(OUT_DIR, "region.csv"), index=False)
